In [1]:
!pip list | grep scikit
!pip install scikit-learn==0.23.1

scikit-learn          0.23.1
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [11]:
import lightgbm as lgb
from sklearn.inspection import permutation_importance
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error, make_scorer
import gc
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from IPython.display import display

from sklearn.metrics import f1_score

In [12]:
df_mfccs = pd.read_pickle('./df_mfccs.pkl')
y = df_mfccs['ebird_code'].values.astype('int')
df_mfccs = df_mfccs.loc[:,'mfcc_2':'mfcc_13']
df_mfccs.head()

,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,mfcc_12,mfcc_13
0,16.952284,-92.89083,8.931996,-1.9169099,-5.08037,-6.159536,-3.9676294,-6.1450853,-5.1676064,-15.280458,-5.044963,-3.233093
1,91.175545,-37.70263,30.003351,-6.070588,19.513779,-21.771854,21.148273,-7.026763,15.1568165,-9.547502,5.46804,1.8762405
2,90.87377,-37.172863,29.778658,-5.463961,19.334085,-21.389166,21.019709,-6.6005473,15.075856,-9.305592,5.4286046,2.102446
3,78.100494,-43.94609,37.06793,-5.7614403,13.569123,-23.764679,25.742575,-6.4576793,6.9876895,-7.653295,8.499181,-1.497807
4,77.596695,-43.481487,36.7084,-5.1844587,13.202621,-23.426989,25.53736,-6.0791764,6.745213,-7.429431,8.41996,-1.3411825


In [16]:
df_mfccs_min = pd.read_pickle('./df_mfccs_min.pkl')
df_mfccs_min = df_mfccs_min.loc[:,'mfcc_min_2':'mfcc_min_13']
# df_mfccs_min = pd.DataFrame({"importances_mean":result["importances_mean"], "importances_std":result["importances_std"]}, index=df_mfccs.columns)
#     perm_imp_df.to_csv('permi.csv')
#     perm_imp_list += [perm_imp_df]
#     print("[Permutation feature Importance]")
#     display(perm_imp_df)
#     perm_imp_df.sort_values("importances_mean", ascending=False).importances_mean.plot.barh()
#     plt.show
# df_mfccs_min.columns = [colum + '_min' for colum in df_mfccs_min.columns]
df_mfccs = pd.concat([df_mfccs, df_mfccs_min], axis=1)
df_mfccs

,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,...,mfcc_min_4,mfcc_min_5,mfcc_min_6,mfcc_min_7,mfcc_min_8,mfcc_min_9,mfcc_min_10,mfcc_min_11,mfcc_min_12,mfcc_min_13
0,16.952284,-92.89083,8.931996,-1.9169099,-5.08037,-6.159536,-3.9676294,-6.1450853,-5.1676064,-15.280458,...,-34.332756,-75.23479,-50.76887,-53.71623,-56.6391,-53.77175,-45.697395,-63.436707,-41.26149,-41.137077
1,91.175545,-37.70263,30.003351,-6.070588,19.513779,-21.771854,21.148273,-7.026763,15.1568165,-9.547502,...,-36.088326,-58.69844,-46.454987,-73.01481,-26.062017,-47.55182,-37.85633,-59.67534,-39.51478,-42.084892
2,90.87377,-37.172863,29.778658,-5.463961,19.334085,-21.389166,21.019709,-6.6005473,15.075856,-9.305592,...,-36.434486,-58.49034,-46.92379,-72.731,-26.49289,-47.636177,-38.251286,-59.697006,-40.083633,-41.747665
3,78.100494,-43.94609,37.06793,-5.7614403,13.569123,-23.764679,25.742575,-6.4576793,6.9876895,-7.653295,...,-34.14868,-71.27188,-44.78651,-81.38721,-29.027485,-55.147526,-47.68649,-56.48716,-39.594505,-50.522804
4,77.596695,-43.481487,36.7084,-5.1844587,13.202621,-23.426989,25.53736,-6.0791764,6.745213,-7.429431,...,-35.042866,-72.10492,-45.891075,-81.40259,-29.547817,-56.06847,-47.982746,-56.523468,-40.17074,-50.812412
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31271,39.472927,-63.920097,-23.938093,-21.633509,-21.639513,-26.608538,-9.614115,-5.8601837,-0.33767584,-5.359286,...,-69.19409,-72.770355,-64.73473,-99.85355,-53.51421,-46.1187,-62.269066,-67.09468,-39.64475,-49.267326
31272,181.55719,1.8581733,-40.133556,-20.272554,-2.436641,-15.80855,4.5602255,-6.5526342,0.74740106,-7.025757,...,-78.452675,-61.854706,-73.1762,-77.30301,-36.193928,-52.745796,-66.95566,-59.87191,-21.550642,-52.918858
31273,180.98882,2.9608285,-38.784187,-20.21594,-2.3900573,-15.215766,4.6777177,-5.8534536,0.71840125,-6.3061705,...,-76.6672,-62.19957,-75.76448,-77.34551,-33.028366,-52.929184,-69.523796,-59.24694,-21.055473,-52.580322
31274,108.49365,-22.353409,37.120758,-14.78723,10.731105,-15.555555,9.360824,-7.8737984,3.4842594,-10.224251,...,-2.4819303,-68.75113,-49.380966,-74.12945,-29.806274,-44.9106,-59.449245,-59.668617,-28.614239,-47.12674


In [17]:
df_mfccs_max = pd.read_pickle('./df_mfccs_max.pkl')
df_mfccs_max = df_mfccs_max.loc[:,'mfcc_max_2':'mfcc_max_13']
# df_mfccs_max.columns = [colum + '_min' for colum in df_mfccs_min.columns]
df_mfccs = pd.concat([df_mfccs, df_mfccs_max], axis=1)
df_mfccs

,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,...,mfcc_max_4,mfcc_max_5,mfcc_max_6,mfcc_max_7,mfcc_max_8,mfcc_max_9,mfcc_max_10,mfcc_max_11,mfcc_max_12,mfcc_max_13
0,16.952284,-92.89083,8.931996,-1.9169099,-5.08037,-6.159536,-3.9676294,-6.1450853,-5.1676064,-15.280458,...,103.77966,39.436943,42.457825,47.78163,33.70136,37.56938,37.706856,22.843487,48.088905,35.936035
1,91.175545,-37.70263,30.003351,-6.070588,19.513779,-21.771854,21.148273,-7.026763,15.1568165,-9.547502,...,93.26336,71.89667,70.99249,20.109034,81.63905,40.172424,57.862778,36.768814,50.762005,53.221825
2,90.87377,-37.172863,29.778658,-5.463961,19.334085,-21.389166,21.019709,-6.6005473,15.075856,-9.305592,...,93.227066,73.195,71.183334,20.438457,83.957756,40.86443,57.649017,36.588326,51.537254,53.80084
3,78.100494,-43.94609,37.06793,-5.7614403,13.569123,-23.764679,25.742575,-6.4576793,6.9876895,-7.653295,...,108.55419,74.34181,75.89072,26.124718,88.72747,44.525635,60.725246,38.34871,56.18856,48.277992
4,77.596695,-43.481487,36.7084,-5.1844587,13.202621,-23.426989,25.53736,-6.0791764,6.745213,-7.429431,...,108.51758,75.30836,75.528,27.348627,88.99898,44.88535,60.518288,38.303658,56.373634,48.730194
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31271,39.472927,-63.920097,-23.938093,-21.633509,-21.639513,-26.608538,-9.614115,-5.8601837,-0.33767584,-5.359286,...,34.62342,58.604355,24.20286,10.360683,42.08709,53.023506,53.847557,53.201797,60.431793,46.480766
31272,181.55719,1.8581733,-40.133556,-20.272554,-2.436641,-15.80855,4.5602255,-6.5526342,0.74740106,-7.025757,...,34.98222,51.546288,41.40689,23.7786,66.92499,61.71364,36.29089,55.79315,68.016655,58.31201
31273,180.98882,2.9608285,-38.784187,-20.21594,-2.3900573,-15.215766,4.6777177,-5.8534536,0.71840125,-6.3061705,...,37.198822,55.08892,41.41618,28.15549,68.16574,62.82563,37.24814,56.48446,69.77184,57.81807
31274,108.49365,-22.353409,37.120758,-14.78723,10.731105,-15.555555,9.360824,-7.8737984,3.4842594,-10.224251,...,97.599525,57.06743,50.7222,35.65425,77.95543,40.730854,45.705574,38.876595,55.704063,36.679485


In [18]:
df_mfccs_std = pd.read_pickle('./df_mfccs_std.pkl')
df_mfccs_std = df_mfccs_std.loc[:,'mfcc_std_2':'mfcc_std_13']
# df_mfccs_max.columns = [colum + '_min' for colum in df_mfccs_min.columns]
df_mfccs = pd.concat([df_mfccs, df_mfccs_std], axis=1)
df_mfccs

,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,...,mfcc_std_4,mfcc_std_5,mfcc_std_6,mfcc_std_7,mfcc_std_8,mfcc_std_9,mfcc_std_10,mfcc_std_11,mfcc_std_12,mfcc_std_13
0,16.952284,-92.89083,8.931996,-1.9169099,-5.08037,-6.159536,-3.9676294,-6.1450853,-5.1676064,-15.280458,...,18.024136,13.695109,11.099782,12.470149,11.406123,10.790547,10.557825,10.982779,9.72156,9.636087
1,91.175545,-37.70263,30.003351,-6.070588,19.513779,-21.771854,21.148273,-7.026763,15.1568165,-9.547502,...,15.05446,16.877956,14.09436,13.9768305,12.63197,12.724309,11.770321,12.629679,11.081962,12.146799
2,90.87377,-37.172863,29.778658,-5.463961,19.334085,-21.389166,21.019709,-6.6005473,15.075856,-9.305592,...,15.102662,16.976788,14.225188,14.089258,12.706847,12.807666,11.833867,12.723135,11.138651,12.204339
3,78.100494,-43.94609,37.06793,-5.7614403,13.569123,-23.764679,25.742575,-6.4576793,6.9876895,-7.653295,...,17.54051,19.501509,15.945928,14.329156,14.691611,12.547157,13.0516615,12.134915,11.378702,11.533379
4,77.596695,-43.481487,36.7084,-5.1844587,13.202621,-23.426989,25.53736,-6.0791764,6.745213,-7.429431,...,17.597902,19.637499,16.080626,14.435461,14.7880945,12.624737,13.1449,12.212531,11.44076,11.595125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31271,39.472927,-63.920097,-23.938093,-21.633509,-21.639513,-26.608538,-9.614115,-5.8601837,-0.33767584,-5.359286,...,9.726189,16.625288,9.570489,13.9472,9.953662,11.197014,9.618289,9.911705,8.94214,8.773064
31272,181.55719,1.8581733,-40.133556,-20.272554,-2.436641,-15.80855,4.5602255,-6.5526342,0.74740106,-7.025757,...,15.240105,17.511913,15.333162,15.689777,15.35891,13.489256,14.128905,12.164888,12.339319,11.305353
31273,180.98882,2.9608285,-38.784187,-20.21594,-2.3900573,-15.215766,4.6777177,-5.8534536,0.71840125,-6.3061705,...,15.289885,17.698738,15.398339,16.0349,15.431085,13.775043,14.34939,12.400831,12.489139,11.517611
31274,108.49365,-22.353409,37.120758,-14.78723,10.731105,-15.555555,9.360824,-7.8737984,3.4842594,-10.224251,...,12.692302,14.474895,11.946087,11.710023,12.419499,10.141437,11.061408,9.785125,9.6402025,9.137571


In [19]:
df_centroids = pd.read_pickle('./df_centroids.pkl')
df_centroids = df_centroids.loc[:,'centroid_mean':'centroid_std']
# df_mfccs_max.columns = [colum + '_min' for colum in df_mfccs_min.columns]
df_mfccs = pd.concat([df_mfccs, df_centroids], axis=1)
df_mfccs

,mfcc_2,mfcc_3,mfcc_4,mfcc_5,mfcc_6,mfcc_7,mfcc_8,mfcc_9,mfcc_10,mfcc_11,...,mfcc_std_8,mfcc_std_9,mfcc_std_10,mfcc_std_11,mfcc_std_12,mfcc_std_13,centroid_mean,centroid_max,centroid_min,centroid_std
0,16.952284,-92.89083,8.931996,-1.9169099,-5.08037,-6.159536,-3.9676294,-6.1450853,-5.1676064,-15.280458,...,11.406123,10.790547,10.557825,10.982779,9.72156,9.636087,5251.498470784159,13342.152954392304,3583.638635963939,1160.9133226262868
1,91.175545,-37.70263,30.003351,-6.070588,19.513779,-21.771854,21.148273,-7.026763,15.1568165,-9.547502,...,12.63197,12.724309,11.770321,12.629679,11.081962,12.146799,4488.815651941392,11049.433100784167,0.0,752.5483820829526
2,90.87377,-37.172863,29.778658,-5.463961,19.334085,-21.389166,21.019709,-6.6005473,15.075856,-9.305592,...,12.706847,12.807666,11.833867,12.723135,11.138651,12.204339,4467.524035969087,9876.209483592975,0.0,763.0639496576156
3,78.100494,-43.94609,37.06793,-5.7614403,13.569123,-23.764679,25.742575,-6.4576793,6.9876895,-7.653295,...,14.691611,12.547157,13.0516615,12.134915,11.378702,11.533379,4866.797575802578,11025.00047745416,0.0,832.1938695867539
4,77.596695,-43.481487,36.7084,-5.1844587,13.202621,-23.426989,25.53736,-6.0791764,6.745213,-7.429431,...,14.7880945,12.624737,13.1449,12.212531,11.44076,11.595125,4859.822736864559,11037.39870771173,0.0,843.2876354415589
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31271,39.472927,-63.920097,-23.938093,-21.633509,-21.639513,-26.608538,-9.614115,-5.8601837,-0.33767584,-5.359286,...,9.953662,11.197014,9.618289,9.911705,8.94214,8.773064,5834.2157182603305,15420.886349232387,2764.3236503750245,1039.4984650326044
31272,181.55719,1.8581733,-40.133556,-20.272554,-2.436641,-15.80855,4.5602255,-6.5526342,0.74740106,-7.025757,...,15.35891,13.489256,14.128905,12.164888,12.339319,11.305353,2152.280592703229,11436.58043991903,1188.5015146453832,505.60544717730556
31273,180.98882,2.9608285,-38.784187,-20.21594,-2.3900573,-15.215766,4.6777177,-5.8534536,0.71840125,-6.3061705,...,15.431085,13.775043,14.34939,12.400831,12.489139,11.517611,2141.3247262166083,11010.306421984435,1138.1274681839711,515.8132277089197
31274,108.49365,-22.353409,37.120758,-14.78723,10.731105,-15.555555,9.360824,-7.8737984,3.4842594,-10.224251,...,12.419499,10.141437,11.061408,9.785125,9.6402025,9.137571,4035.2487960972026,11097.066687103103,0.0,752.398700801798


In [20]:

X = df_mfccs.values.astype('float32')
del df_mfccs
gc.collect()

911

https://blog.amedama.jp/entry/lightgbm-custom-metric

In [21]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred)) 
mse_scorer = make_scorer(rmse)

In [22]:
n_splits = 4
skf = StratifiedKFold(n_splits=n_splits)
for i, (train_index, valid_index) in enumerate(skf.split(X, y)):
    train_X, train_y = X[train_index], y[train_index]
    valid_X, valid_y = X[valid_index], y[valid_index]
    
    
    model = lgb.LGBMClassifier(objective='multiclass',
                               learning_rate=0.05,
                               n_estimators=100
                              )
    model.fit(train_X, train_y,
            eval_set=[(valid_X, valid_y), (train_X, train_y)],
            eval_metric='multi_logloss',
            verbose=200,
            early_stopping_rounds=100)
    valid_pred = model.predict_proba(valid_X, num_iteration=model.best_iteration_)
   
    
    print(str(i+1)+'/'+str(n_splits)+'fold end.')
    print(f1_score(valid_y, np.argmax(valid_pred, axis = 1), average='micro'))
    
    result = permutation_importance(model, valid_X, valid_y, scoring=mse_scorer,n_repeats=10, n_jobs=-1, random_state=71)
#     perm_imp_df = pd.DataFrame({"importances_mean":result["importances_mean"], "importances_std":result["importances_std"]}, index=df_mfccs.columns)
#     perm_imp_df.to_csv('permi.csv')
#     perm_imp_list += [perm_imp_df]
#     print("[Permutation feature Importance]")
#     display(perm_imp_df)
#     perm_imp_df.sort_values("importances_mean", ascending=False).importances_mean.plot.barh()
#     plt.show()
    
    break
    
 

Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	training's multi_logloss: 0.0663409	valid_0's multi_logloss: 4.6978
1/4fold end.
0.10858166005883105


KeyboardInterrupt: 

In [ ]:
valid_pred.shape

In [ ]:
np.argmax(valid_pred, axis = 1)

In [ ]:
valid_y.astype('int')

In [ ]:
X

In [ ]:
valid_pred